In [1]:
import os
from google.api_core.retry import Retry
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, storage
from oauth2client.service_account import ServiceAccountCredentials
# import pandas as pd
import cProfile
# import dask.dataframe as dd
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType

import shutil  # 파일 복사를 위한 라이브러리
import glob    # 파일들을 패턴 매칭으로 검색할 때 사용하는 라이브러리

In [2]:
# Firebase 초기화
try:
    cred = credentials.Certificate('C:/Users/admin/Desktop/voice_data_queries/samboss-reward-firebase-adminsdk-9j6t2-4f53253105.json')
    firebase_admin.initialize_app(cred)
    bucket = storage.bucket("samboss-reward.appspot.com")
    db = firestore.client()
    print("firebase_admin 권한 확인 성공.")
except Exception as e:
    print("firebase_admin 권한 확인 실패.")
    print(f"Exception type: {type(e)}")
    print(f"Exception message: {str(e)}")

firebase_admin 권한 확인 성공.


In [3]:
# 가상 환경의 Python 경로를 지정
python_path = "C:/Users/admin/anaconda3/envs/my_conda_01/python.exe"

# Spark 세션 생성
spark = SparkSession.builder \
    .appName("voice_metadata integrating operation") \
    .config("spark.driver.memory", "14g") \
    .config("spark.executor.memory", "14g")  \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.pyspark.python", python_path) \
    .config("spark.local.dir", "D:/spark_tmp") \
    .getOrCreate()

# SparkContext 가져오기
sc = spark.sparkContext

 Firebase bucket의 모든 파일과 폴더 정보를 포함할 spark DF

In [ ]:
schema = StructType([
    StructField("full_path", StringType(), True),
    StructField("directory", StringType(), True),
    StructField("file_name", StringType(), True)
])

firebase_metadata_sdf = spark.createDataFrame([], schema)

# Firebase 버킷 내의 모든 blob을 가져옴
blobs = bucket.list_blobs()

data = []
# Spark DataFrame에 blob 메타데이터 채우기
for blob in blobs:
    full_path = blob.name
    directory = '/'.join(full_path.split('/')[:-1])
    file_name = full_path.split('/')[-1]
    
    data.append((full_path, directory, file_name))

# Spark DataFrame 업데이트
firebase_metadata_sdf = spark.createDataFrame(data, ["full_path", "directory", "file_name"])

# 나중에 사용하기 위해 Spark DataFrame을 parquet 파일로 저장 (또는 다른 포맷도 가능)
firebase_metadata_sdf.write.parquet("firebase_metadata.parquet")

# 데이터 확인
firebase_metadata_sdf.show()

In [ ]:
# DataFrame에서 검색 수행
def search_in_df(df, user_id, ad_name):
    # 'user_id'와 'ad_name'에 해당하는 row를 필터링
    filtered_df = df[
        (df['user_id'] == user_id) & 
        (df['blob_name'].str.endswith(ad_name)) & 
        (~df['blob_name'].str.endswith('.txt'))
    ]
    
    return filtered_df